In [1]:
from gurobipy import *
import snap
from graphviz import Digraph
import pandas as pd
import numpy as np
import time

In [2]:
# G1 = snap.TUNGraph.New()
# G1.AddNode(0)
# G1.AddNode(1)
# G1.AddNode(2)
# G1.AddNode(3)
# G1.AddNode(4)
# G1.AddNode(5)
# G1.AddNode(6)
# G1.AddNode(7)
# G1.AddNode(8)
# G1.AddNode(9)
# G1.AddNode(10)
# G1.AddNode(11)
# G1.AddNode(12)
# G1.AddNode(13)
# G1.AddNode(14)
# G1.AddEdge(0,1)
# G1.AddEdge(0,2)
# G1.AddEdge(2,3)
# G1.AddEdge(1,3)
# G1.AddEdge(3,4)
# G1.AddEdge(4,5)
# G1.AddEdge(4,6)
# G1.AddEdge(0,7)
# G1.AddEdge(7,8)
# G1.AddEdge(7,9)
# G1.AddEdge(9,10)
# G1.AddEdge(0,11)
# G1.AddEdge(11,12)
# G1.AddEdge(11,13)
# G1.AddEdge(13,14)
# G1

In [3]:
# G1 = snap.GenRndGnm(snap.TUNGraph, 360, 900)
G1 = snap.LoadEdgeList(snap.TUNGraph, "graph.txt", 0, 1)

In [4]:
#參數設定
Budget = 2
init_S = 0

In [5]:
#刪除孤單的點
lonely_node = []
for NI in G1.Nodes():
    if NI.GetOutDeg()==0:
        lonely_node.append(NI.GetId())
        G1.DelNode(NI.GetId())

print("lonely_node = ",lonely_node)

lonely_node =  []


In [6]:
AllnodeID = []
InfectednodeID = [init_S]
V = [] #AllnodeID\0(s) #vulnerablenodeID
vaccinatednodeID = [] #vaccinated的node
for NI in G1.Nodes():
    AllnodeID.append(NI.GetId())
    if(NI.GetId()!=init_S):
        V.append(NI.GetId())

In [7]:
layer = 0
for NI in G1.Nodes():
    if(G1.GetShortPath(NI.GetId(),init_S)>layer):
        layer = G1.GetShortPath(NI.GetId(),init_S)
layer

7

# IP Programming

In [8]:
B = Budget #budget
S = init_S #初始感染點
#Create empty model
IP_model = Model('IP Vaccination Strategy')

#add variablies
#S = model.addVars([(u,t) for u in V for t in range(1,layer+1)],vtype=GRB.BINARY, name="S")
X = IP_model.addVars([(u,t) for u in V for t in range(1,layer+1) ],vtype=GRB.BINARY, name="X")
Y = IP_model.addVars([(v) for v in V],vtype=GRB.BINARY, name="Y")
IP_model.update()
#add objective function
# obj = 0
# for v in V:
#     obj = obj+Y[v]
IP_model.setObjective(quicksum(Y[v] for v in V) ,GRB.MAXIMIZE)

#constraint 1
IP_model.addConstrs(quicksum(X[u,t] for u in V)<=B for t in range(1,layer+1))
#constraint 2
#model.addConstrs(quicksum(X[u,t] for t in range(1,layer+1))<=1 for u in V) 
#constraint 3
IP_model.addConstrs(quicksum(X[u,t] for u in V for t in range(1,(G1.GetShortPath(S,v)-G1.GetShortPath(u,v))+1) if t>=0 )>= Y[v]  for v in V)
IP_model.update()
IP_model.optimize()

Academic license - for non-commercial use only - expires 2022-10-03
Using license file C:\Users\user\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2201 rows, 17552 columns and 1526342 nonzeros
Model fingerprint: 0x6d48bf0f
Variable types: 0 continuous, 17552 integer (17552 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 6 rows and 5998 columns
Presolve time: 4.01s
Presolved: 2195 rows, 11554 columns, 1509191 nonzeros
Variable types: 0 continuous, 11554 integer (11538 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
     961    2.0877190e+03   1.581903e+04   0.000000e+00      5s
    8073    1.6845808e+03   6.005655e+02   0.000000e+00     10s
    

     0     0 1652.67633    0 1193 1569.00000 1652.67633  5.33%     -   53s
     0     0 1652.66848    0 1196 1569.00000 1652.66848  5.33%     -   53s
     0     0 1652.66809    0 1197 1569.00000 1652.66809  5.33%     -   53s
     0     0 1652.40068    0 1190 1569.00000 1652.40068  5.32%     -   54s
     0     0 1652.36352    0 1193 1569.00000 1652.36352  5.31%     -   55s
     0     0 1652.35210    0 1200 1569.00000 1652.35210  5.31%     -   55s
     0     0 1652.16756    0 1196 1569.00000 1652.16756  5.30%     -   56s
     0     0 1652.14437    0 1201 1569.00000 1652.14437  5.30%     -   57s
     0     0 1652.13745    0 1200 1569.00000 1652.13745  5.30%     -   57s
     0     0 1652.13233    0 1198 1569.00000 1652.13233  5.30%     -   57s
     0     0 1652.13161    0 1198 1569.00000 1652.13161  5.30%     -   57s
     0     0 1651.99331    0 1200 1569.00000 1651.99331  5.29%     -   58s
     0     0 1651.96350    0 1208 1569.00000 1651.96350  5.29%     -   58s
     0     0 1651.95875  

  2819   990 1599.20658   25  996 1571.00000 1615.08440  2.81%   260  301s
  3005  1066 1610.20421   22  986 1571.00000 1614.23226  2.75%   258  306s
  3213  1167     cutoff   48      1571.00000 1612.80470  2.66%   253  312s
  3317  1220 1587.19122   31 1014 1571.00000 1612.37829  2.63%   252  315s
  3534  1357     cutoff   39      1571.00000 1611.88905  2.60%   250  322s
  3662  1416 1588.09144   22 1049 1571.00000 1611.41310  2.57%   249  326s
  3789  1473 1585.19139   29 1030 1571.00000 1610.95455  2.54%   248  330s
  4040  1590 1586.96595   27 1043 1571.00000 1610.14594  2.49%   247  338s
  4174  1661 1596.76673   24 1023 1571.00000 1609.45127  2.45%   246  344s
  4328  1720 1589.72637   28 1023 1571.00000 1608.95692  2.42%   246  347s
  4462  1728 1597.94498   22  985 1571.00000 1608.48721  2.39%   246  357s
  4498  1779 1584.73602   34 1000 1571.00000 1608.32978  2.38%   245  361s
  4647  1844     cutoff   30      1571.00000 1608.05203  2.36%   245  366s
  4802  1923 1585.15576  

In [9]:
print("IP Programming")
print("保護的個數=",IP_model.ObjVal)
for var in IP_model.getVars():
    if(var.x>0):
        print(var.varName,'\t', var.x)

IP Programming
保護的個數= 1571.0
X[1270,1] 	 1.0
X[337,1] 	 1.0
X[1474,2] 	 1.0
X[78,2] 	 1.0
X[1999,3] 	 1.0
X[2152,4] 	 1.0
X[711,7] 	 1.0
X[1988,5] 	 1.0
X[260,6] 	 1.0
X[1329,4] 	 1.0
X[927,3] 	 1.0
X[894,7] 	 1.0
X[1132,5] 	 1.0
X[1288,6] 	 1.0
Y[1270] 	 1.0
Y[1] 	 1.0
Y[14] 	 1.0
Y[440] 	 1.0
Y[1763] 	 1.0
Y[633] 	 1.0
Y[1088] 	 1.0
Y[1526] 	 1.0
Y[3] 	 1.0
Y[474] 	 1.0
Y[1290] 	 1.0
Y[1983] 	 1.0
Y[5] 	 1.0
Y[9] 	 1.0
Y[481] 	 1.0
Y[490] 	 1.0
Y[681] 	 1.0
Y[693] 	 1.0
Y[759] 	 1.0
Y[1519] 	 1.0
Y[1757] 	 1.0
Y[2059] 	 1.0
Y[1173] 	 1.0
Y[1277] 	 1.0
Y[2089] 	 1.0
Y[7] 	 1.0
Y[758] 	 1.0
Y[815] 	 1.0
Y[1017] 	 1.0
Y[1399] 	 1.0
Y[1843] 	 1.0
Y[2182] 	 1.0
Y[8] 	 1.0
Y[115] 	 1.0
Y[605] 	 1.0
Y[614] 	 1.0
Y[777] 	 1.0
Y[1133] 	 1.0
Y[1779] 	 1.0
Y[2137] 	 1.0
Y[283] 	 1.0
Y[337] 	 1.0
Y[1344] 	 1.0
Y[2084] 	 1.0
Y[2129] 	 1.0
Y[10] 	 1.0
Y[588] 	 1.0
Y[1150] 	 1.0
Y[1557] 	 1.0
Y[1711] 	 1.0
Y[2015] 	 1.0
Y[11] 	 1.0
Y[230] 	 1.0
Y[309] 	 1.0
Y[810] 	 1.0
Y[938] 	 1.0
Y[1084] 	 1.0
Y

Y[226] 	 1.0
Y[1289] 	 1.0
Y[2022] 	 1.0
Y[374] 	 1.0
Y[521] 	 1.0
Y[701] 	 1.0
Y[1333] 	 1.0
Y[1857] 	 1.0
Y[827] 	 1.0
Y[849] 	 1.0
Y[861] 	 1.0
Y[836] 	 1.0
Y[657] 	 1.0
Y[763] 	 1.0
Y[1941] 	 1.0
Y[1978] 	 1.0
Y[2023] 	 1.0
Y[2103] 	 1.0
Y[502] 	 1.0
Y[1657] 	 1.0
Y[1669] 	 1.0
Y[2185] 	 1.0
Y[325] 	 1.0
Y[568] 	 1.0
Y[235] 	 1.0
Y[2000] 	 1.0
Y[238] 	 1.0
Y[930] 	 1.0
Y[239] 	 1.0
Y[1453] 	 1.0
Y[1811] 	 1.0
Y[317] 	 1.0
Y[1033] 	 1.0
Y[1643] 	 1.0
Y[1919] 	 1.0
Y[631] 	 1.0
Y[1441] 	 1.0
Y[1481] 	 1.0
Y[1614] 	 1.0
Y[1705] 	 1.0
Y[1283] 	 1.0
Y[1671] 	 1.0
Y[1209] 	 1.0
Y[1472] 	 1.0
Y[1787] 	 1.0
Y[1832] 	 1.0
Y[1582] 	 1.0
Y[1951] 	 1.0
Y[2118] 	 1.0
Y[1153] 	 1.0
Y[1220] 	 1.0
Y[2086] 	 1.0
Y[247] 	 1.0
Y[1139] 	 1.0
Y[1928] 	 1.0
Y[728] 	 1.0
Y[800] 	 1.0
Y[1939] 	 1.0
Y[355] 	 1.0
Y[505] 	 1.0
Y[2026] 	 1.0
Y[690] 	 1.0
Y[1246] 	 1.0
Y[252] 	 1.0
Y[271] 	 1.0
Y[944] 	 1.0
Y[1518] 	 1.0
Y[1551] 	 1.0
Y[253] 	 1.0
Y[1262] 	 1.0
Y[254] 	 1.0
Y[559] 	 1.0
Y[941] 	 1.0
Y[1260] 	 

Y[737] 	 1.0
Y[1022] 	 1.0
Y[1119] 	 1.0
Y[904] 	 1.0
Y[1439] 	 1.0
Y[1632] 	 1.0
Y[1342] 	 1.0
Y[838] 	 1.0
Y[1743] 	 1.0
Y[1798] 	 1.0
Y[749] 	 1.0
Y[767] 	 1.0
Y[1922] 	 1.0
Y[756] 	 1.0
Y[1970] 	 1.0
Y[760] 	 1.0
Y[1667] 	 1.0
Y[1926] 	 1.0
Y[761] 	 1.0
Y[762] 	 1.0
Y[985] 	 1.0
Y[766] 	 1.0
Y[878] 	 1.0
Y[1406] 	 1.0
Y[1297] 	 1.0
Y[1241] 	 1.0
Y[1316] 	 1.0
Y[887] 	 1.0
Y[780] 	 1.0
Y[781] 	 1.0
Y[1047] 	 1.0
Y[1622] 	 1.0
Y[782] 	 1.0
Y[868] 	 1.0
Y[792] 	 1.0
Y[1595] 	 1.0
Y[1331] 	 1.0
Y[1703] 	 1.0
Y[1932] 	 1.0
Y[820] 	 1.0
Y[1576] 	 1.0
Y[1956] 	 1.0
Y[2024] 	 1.0
Y[1349] 	 1.0
Y[1348] 	 1.0
Y[1649] 	 1.0
Y[2140] 	 1.0
Y[1169] 	 1.0
Y[819] 	 1.0
Y[1776] 	 1.0
Y[899] 	 1.0
Y[831] 	 1.0
Y[1550] 	 1.0
Y[1692] 	 1.0
Y[2119] 	 1.0
Y[1532] 	 1.0
Y[1714] 	 1.0
Y[1286] 	 1.0
Y[1953] 	 1.0
Y[848] 	 1.0
Y[1171] 	 1.0
Y[1214] 	 1.0
Y[1148] 	 1.0
Y[864] 	 1.0
Y[1029] 	 1.0
Y[1904] 	 1.0
Y[1566] 	 1.0
Y[893] 	 1.0
Y[871] 	 1.0
Y[1959] 	 1.0
Y[1102] 	 1.0
Y[1468] 	 1.0
Y[1810] 	 1.0
Y[89

# LP Programming

In [10]:
B = Budget #budget
S = init_S #初始感染點
#Create empty model
LP_model = Model('LP Vaccination Strategy')

#add variablies
#S = model.addVars([(u,t) for u in V for t in range(1,layer+1)],vtype=GRB.BINARY, name="S")
X = LP_model.addVars([(u,t) for u in V for t in range(1,layer+1) ],lb=0,ub=1, name="X")
Y = LP_model.addVars([(v) for v in V],lb=0,ub=1, name="Y")
LP_model.update()
#add objective function
# obj = 0
# for v in V:
#     obj = obj+Y[v]
LP_model.setObjective(quicksum(Y[v] for v in V) ,GRB.MAXIMIZE)

#constraint 1
LP_model.addConstrs(quicksum(X[u,t] for u in V)<=B for t in range(1,layer+1))
#constraint 3
LP_model.addConstrs(quicksum(X[u,t] for u in V for t in range(1,(G1.GetShortPath(S,v)-G1.GetShortPath(u,v))+1) if t>=0 )>= Y[v]  for v in V)
LP_model.update()
LP_model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2201 rows, 17552 columns and 1526342 nonzeros
Model fingerprint: 0xdfb8078a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 7 rows and 5553 columns
Presolve time: 0.73s
Presolved: 2194 rows, 11999 columns, 1520585 nonzeros

Ordering time: 0.12s

Barrier statistics:
 AA' NZ     : 2.347e+06
 Factor NZ  : 2.404e+06 (roughly 25 MBytes of memory)
 Factor Ops : 3.513e+09 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.40147040e+03  1.45753462e+04  5.31e+03 0.00e+00  3.24e+00     3s
   1   2.17326810e

In [11]:
THE_X = [] #取出X[,]
X_Pro = [] #取出X[,]的機率
print("LP Programming")
print("保護的個數=",LP_model.ObjVal)
for var in LP_model.getVars():
    if(var.x>0):
        if(var.varName[0]=="X"):
            THE_X.append(var.varName) 
            X_Pro.append(var.x)
        print(var.varName,'\t', var.x)

LP Programming
保護的個數= 1657.5010075464131
X[188,1] 	 0.3586856567859227
X[912,1] 	 0.16428466804704125
X[1270,1] 	 0.3410226097748903
X[774,3] 	 0.010702093009751251
X[693,1] 	 0.20123973711055615
X[693,2] 	 0.09800198129127147
X[115,2] 	 0.27563523092963826
X[337,1] 	 0.25756699858640153
X[1339,4] 	 0.12334751554221599
X[1754,3] 	 0.09816015524448476
X[2051,5] 	 0.4939711889394375
X[207,4] 	 0.14411423886260344
X[443,1] 	 0.01909051276962162
X[1755,5] 	 0.17173357786015345
X[1474,2] 	 0.4208714214768161
X[78,2] 	 0.16947364041903523
X[1999,3] 	 0.20836363195131138
X[660,1] 	 0.106727983269096
X[803,1] 	 0.09307215419056948
X[1585,5] 	 0.0794102581658927
X[676,2] 	 0.1331673012403438
X[1265,1] 	 0.13081809302015182
X[319,3] 	 0.2602771901025573
X[1019,2] 	 0.010021728334285093
X[679,1] 	 0.01447356674897493
X[2152,4] 	 0.5245321301134606
X[135,2] 	 0.2233112872066307
X[1719,3] 	 0.5352324897100261
X[411,3] 	 0.08337717424182697
X[118,6] 	 0.11466441554074236
X[425,3] 	 0.108592269150532

Y[211] 	 1.0
Y[364] 	 0.4951130748756493
Y[441] 	 0.4752147915613981
Y[993] 	 0.9402852158056078
Y[1040] 	 0.16428466804704125
Y[1131] 	 0.44893321419006704
Y[1572] 	 1.0
Y[69] 	 1.0
Y[242] 	 1.0
Y[319] 	 0.289389431206464
Y[687] 	 1.0
Y[850] 	 1.0
Y[1310] 	 0.3961486159129062
Y[1619] 	 1.0
Y[1744] 	 0.8440274815326494
Y[2110] 	 0.16428466804704125
Y[2158] 	 1.0
Y[70] 	 1.0
Y[656] 	 0.6442724128158173
Y[1483] 	 0.4400083030491606
Y[71] 	 0.47733042235479034
Y[1675] 	 0.7435172513613797
Y[1884] 	 1.0
Y[1931] 	 0.05132766449441138
Y[2091] 	 1.0
Y[72] 	 0.21126146544484126
Y[939] 	 0.21126146544484126
Y[1019] 	 0.010021728334285093
Y[1357] 	 1.0
Y[1852] 	 0.4278553432028103
Y[2101] 	 0.7593918493178587
Y[573] 	 0.1912830117707423
Y[698] 	 0.5426328639539328
Y[1845] 	 0.19652181977183103
Y[74] 	 0.8146322607261443
Y[817] 	 1.0
Y[255] 	 1.0
Y[420] 	 0.5567359136344214
Y[679] 	 1.0
Y[702] 	 1.0
Y[784] 	 0.4519255187022281
Y[1647] 	 0.8851340751763966
Y[1925] 	 1.0
Y[76] 	 0.38886004681949016

Y[663] 	 1.0
Y[2041] 	 1.0
Y[160] 	 0.8542433521437552
Y[1368] 	 1.0
Y[1409] 	 1.0
Y[413] 	 0.9871333072743553
Y[713] 	 0.5281592972049579
Y[924] 	 0.6831073684794418
Y[162] 	 1.0
Y[523] 	 1.0
Y[547] 	 1.0
Y[611] 	 1.0
Y[1094] 	 0.3601131225445119
Y[1207] 	 1.0
Y[1673] 	 1.0
Y[1307] 	 1.0
Y[1514] 	 0.5117447410410705
Y[164] 	 0.648489807097114
Y[1588] 	 0.7545174914391058
Y[1600] 	 1.0
Y[2048] 	 0.6162526553723242
Y[243] 	 1.0
Y[1101] 	 0.49446131151775097
Y[1579] 	 0.3410226097748903
Y[1794] 	 0.3410226097748903
Y[1901] 	 0.8311170201240607
Y[394] 	 0.6228578263171991
Y[494] 	 0.01447356674897493
Y[1538] 	 0.10549162631425224
Y[1554] 	 0.21571330385953108
Y[2166] 	 0.12120155001807092
Y[167] 	 0.4752147915613981
Y[950] 	 0.4752147915613981
Y[2151] 	 0.6060328845815499
Y[168] 	 0.8693608818912928
Y[700] 	 0.697163616003911
Y[1356] 	 1.0
Y[1812] 	 1.0
Y[169] 	 1.0
Y[594] 	 1.0
Y[466] 	 1.0
Y[801] 	 0.3969217181938184
Y[1087] 	 0.3586856567859227
Y[1450] 	 1.0
Y[1305] 	 0.905242534839439

Y[1285] 	 1.0
Y[524] 	 0.615857003812642
Y[1048] 	 1.0
Y[1897] 	 1.0
Y[906] 	 1.0
Y[933] 	 0.73525407070419
Y[1020] 	 0.6162526553723242
Y[1466] 	 1.0
Y[1887] 	 1.0
Y[1491] 	 1.0
Y[551] 	 1.0
Y[1726] 	 0.8881792487907277
Y[293] 	 1.0
Y[294] 	 1.0
Y[2044] 	 0.7965809291614695
Y[295] 	 0.3586856567859227
Y[1464] 	 0.3586856567859227
Y[1823] 	 0.6714666322295249
Y[1863] 	 0.4956764341610673
Y[297] 	 1.0
Y[897] 	 0.5014460999503374
Y[2077] 	 1.0
Y[298] 	 1.0
Y[301] 	 1.0
Y[299] 	 1.0
Y[658] 	 1.0
Y[1072] 	 0.8990897847360834
Y[300] 	 1.0
Y[2072] 	 1.0
Y[998] 	 0.3199746401650905
Y[2029] 	 1.0
Y[599] 	 0.10309388252485457
Y[665] 	 0.6591945002266437
Y[775] 	 1.0
Y[881] 	 0.4500352316230031
Y[1569] 	 1.0
Y[682] 	 1.0
Y[1536] 	 1.0
Y[1704] 	 1.0
Y[2167] 	 0.7531320609502742
Y[306] 	 1.0
Y[1376] 	 1.0
Y[822] 	 0.4839949627012819
Y[947] 	 1.0
Y[1045] 	 1.0
Y[311] 	 0.5663658905886929
Y[733] 	 0.07581082568536997
Y[1378] 	 0.23621221484269694
Y[1722] 	 0.8776890655429525
Y[313] 	 1.0
Y[1211] 	 1

Y[544] 	 1.0
Y[1660] 	 0.6176029842441408
Y[1930] 	 1.0
Y[1023] 	 1.0
Y[1683] 	 1.0
Y[2001] 	 1.0
Y[1179] 	 1.0
Y[668] 	 0.7531320609502742
Y[1185] 	 1.0
Y[1242] 	 1.0
Y[990] 	 1.0
Y[1963] 	 1.0
Y[2045] 	 0.7735278338488802
Y[1243] 	 0.7247985557201332
Y[1590] 	 1.0
Y[553] 	 0.3292843690332664
Y[560] 	 0.6881802021679799
Y[1161] 	 0.16499970098622518
Y[1238] 	 1.0
Y[2097] 	 0.47759173439256447
Y[557] 	 0.16428466804704125
Y[953] 	 0.16428466804704125
Y[1808] 	 1.0
Y[1520] 	 1.0
Y[1535] 	 0.3470791554464275
Y[563] 	 0.7795570782627388
Y[565] 	 0.4760260289323994
Y[1082] 	 1.0
Y[1245] 	 0.29596513412717873
Y[1364] 	 1.0
Y[566] 	 0.644895141502852
Y[1380] 	 1.0
Y[1034] 	 0.5941464370125318
Y[1603] 	 1.0
Y[1188] 	 0.5797981093662378
Y[1455] 	 0.8266079790742366
Y[572] 	 0.5054047823136387
Y[785] 	 1.0
Y[1015] 	 0.18229959793481132
Y[1164] 	 0.9677281800114923
Y[574] 	 1.0
Y[1523] 	 1.0
Y[1712] 	 1.0
Y[1130] 	 0.5665660998116647
Y[2163] 	 0.3410226097748903
Y[1681] 	 0.6162526553723242
Y[57

# LP Rounding

In [12]:
def copy_graph(graph):
    tmpfile = '.copy.bin'

    # Saving to tmp file
    FOut = snap.TFOut(tmpfile)
    graph.Save(FOut)
    FOut.Flush()

    # Loading to new graph
    FIn = snap.TFIn(tmpfile)
    graphtype = type(graph)
    new_graph = graphtype.New()
    new_graph = new_graph.Load(FIn)

    return new_graph

In [13]:
start_time = time.time()

In [14]:
import random
timer = 10
count = 0
maximum_saved_nodes = 0 #計算最多被保護的點
Final_strategy = [] #儲存最終最好的策略
print("LP Rounding")
for i in range(timer):
    print("round=",i)
    random_strategy_choose = [] #新策略(格式很亂)
    new_strategy = [[]for i in range(layer)]#新策略(格式清理過)

    for i in range(1,layer+1): #以每一個時間抓出來算機率
        XX = []
        XX_ori = []
        XX_Pro = []

        for x in range(len(THE_X)):
            if THE_X[x][-4]==",":
                if THE_X[x][-3:-1]==str(i):
                    XX.append(THE_X[x])
                    XX_Pro.append(X_Pro[x])
            elif THE_X[x][-2]==str(i):
                XX.append(THE_X[x])
                XX_Pro.append(X_Pro[x])
        if(len(XX)==0): #如果在某時沒有注射
            continue
        XX_ori = XX
#         print("XX_ori",len(XX_ori))
#         print("XX",XX)
#         print("XX_Pro",XX_Pro)
        for i in range(B): #每一個時間挑B個
            if(len(XX_ori)<B):
                random_strategy_choose.append(XX)
                break
            else:
                rdc = random.choices(XX, weights=XX_Pro,k=1) #這次的策略
#                 print("rdc",rdc)
                del XX_Pro[XX.index(rdc[0])]
                XX.remove(rdc[0])

                random_strategy_choose.append(rdc)
#         print("strategy",random_strategy_choose)
    #清理格式    
    for i in range(len(random_strategy_choose)):
        if(random_strategy_choose[i][0][-4]==","): #注射時間是十位數
            new_strategy[int(random_strategy_choose[i][0][-3:-1])-1].append(random_strategy_choose[i][0][2:-4])
        else: #注射時間是個位數
            new_strategy[int(random_strategy_choose[i][0][-2])-1].append(random_strategy_choose[i][0][2:-3])

#     new_strategy = list(map(int, new_strategy)) #str-->int
    print("選擇的策略=",new_strategy)
    ########################################################################開始注射策略
    G_temp = copy_graph(G1)
    S = init_S #初始感染點
    temp_vaccinated = []
    temp_infected = [S]
    temp_vulnerable = V.copy()    
    for i in range(len(new_strategy)): #開始依序注射
        if(len(new_strategy[i])==0):#如果這個時間點沒有注射
            a = temp_vulnerable.copy()
            for n in range(len(a)):#病毒開始擴散
                if(G_temp.GetShortPath(S,a[n])==i+1):
                    temp_infected.append(a[n])
                    temp_vulnerable.remove(a[n])
            continue
        TT_saved = []
        ttemp_saved = []
        for j in range(len(new_strategy[i])):
#             temp_saved = []
            b = temp_vulnerable.copy()
            if(int(new_strategy[i][j]) not in temp_vulnerable): #如果要注射的點已經被保護了(從圖中刪除)
#                 for n in range(len(b)):
#                      if(G_temp.GetShortPath(S,b[n])==i+1):#病毒開始擴散
#                         temp_infected.append(b[n])
#                         temp_vulnerable.remove(b[n])
                continue
                

            for k in range(len(temp_vulnerable)):#每一個點計算是否有被注射的點保護

                r  = i+1
                if(r<=(G_temp.GetShortPath(S,temp_vulnerable[k])-G_temp.GetShortPath(int(new_strategy[i][j]),temp_vulnerable[k]))):
                    ttemp_saved.append(temp_vulnerable[k])
        
        temp_saved = list(dict.fromkeys(ttemp_saved))
        for m in range(len(temp_saved)): #調整位置
            temp_vaccinated.append(temp_saved[m])
            temp_vulnerable.remove(temp_saved[m])
            G_temp.DelNode(temp_saved[m])

        a = temp_vulnerable.copy()
        for n in range(len(a)):#病毒開始擴散
            if(G_temp.GetShortPath(S,a[n])==i+1):
                temp_infected.append(a[n])
                temp_vulnerable.remove(a[n])

            
    print("保護個數=",len(temp_vaccinated))
    if(len(temp_vaccinated)>maximum_saved_nodes):
        maximum_saved_nodes = len(temp_vaccinated)
        Final_strategy = new_strategy
    count = count+int(len(temp_vaccinated))
#     print("被保護的點=",temp_vaccinated)
print("平均被保護個數=",count/timer)
print("Final_strategy = ",Final_strategy)
print("Maximum saved node = ",maximum_saved_nodes)

LP Rounding
round= 0
選擇的策略= [['1265', '1270'], ['1474', '115'], ['927', '1719'], ['581', '207'], ['1755', '1520'], ['1288', '743'], []]
保護個數= 1508
round= 1
選擇的策略= [['912', '337'], ['693', '1562'], ['927', '319'], ['1339', '581'], ['1520', '899'], ['743', '118'], []]
保護個數= 1301
round= 2
選擇的策略= [['396', '1270'], ['115', '1474'], ['314', '1719'], ['2152', '1901'], ['2051', '2040'], ['743', '1288'], []]
保護個數= 1499
round= 3
選擇的策略= [['1849', '1270'], ['115', '693'], ['314', '1999'], ['160', '478'], ['1755', '1520'], ['743', '1288'], []]
保護個數= 1404
round= 4
選擇的策略= [['1270', '396'], ['1562', '693'], ['319', '1754'], ['478', '1339'], ['2051', '2040'], ['743', '1288'], []]
保護個數= 1336
round= 5
選擇的策略= [['660', '188'], ['269', '2061'], ['1799', '927'], ['478', '2152'], ['1520', '223'], ['1288', '743'], []]
保護個數= 1430
round= 6
選擇的策略= [['396', '188'], ['135', '269'], ['1754', '1719'], ['2152', '1368'], ['1755', '1132'], ['1288', '225'], []]
保護個數= 1482
round= 7
選擇的策略= [['693', '912'], ['135', '115'], 

In [15]:
end_time = time.time()
print("計算時間=",end_time-start_time,"sec")

計算時間= 32.083282232284546 sec


# ALG3

In [16]:
start_time = time.time()

In [17]:
import random
timer = 100
count = 0
maximum_saved_nodes = 0 #計算最多被保護的點
Final_strategy = [] #儲存最終最好的策略
print("LP Rounding")
for i in range(timer):
    print("round=",i)
    random_strategy_choose = [] #新策略(格式很亂)
    new_strategy = [[]for i in range(layer)]#新策略(格式清理過)

    for i in range(1,layer+1): #以每一個時間抓出來算機率
        XX = []
        XX_ori = []
        XX_Pro = []

        for x in range(len(THE_X)):
            if THE_X[x][-4]==",":
                if THE_X[x][-3:-1]==str(i):
                    XX.append(THE_X[x])
                    XX_Pro.append(X_Pro[x])
            elif THE_X[x][-2]==str(i):
                XX.append(THE_X[x])
                XX_Pro.append(X_Pro[x])
        if(len(XX)==0): #如果在某時沒有注射
            continue
        XX_ori = XX
#         print("XX_ori",len(XX_ori))#
#         print("XX",XX)#
#         print("XX_Pro",XX_Pro)#

#         for i in range(B): #每一個時間挑B個
#             if(len(XX_ori)<B):
#                 random_strategy_choose.append(XX)
#                 break
#             else:
#                 rdc = random.choices(XX, weights=XX_Pro,k=1) #這次的策略
# #                 print("rdc",rdc)
#                 del XX_Pro[XX.index(rdc[0])]
#                 XX.remove(rdc[0])

#                 random_strategy_choose.append(rdc)
        for i in range(len(XX_ori)): #每一個挑出來根據機率看會不會被選到 #new ALG3
            rand = ['A','B']
            ch_or_not = random.choices(rand, weights = [XX_Pro[i],(1-XX_Pro[i])], k = 1)
            if(ch_or_not[0]=="A"):
                random_strategy_choose.append([XX[i]])
            
                
#     print("strategy",random_strategy_choose)#
    #清理格式    
    for i in range(len(random_strategy_choose)):
        if(random_strategy_choose[i][0][-4]==","): #注射時間是十位數
            new_strategy[int(random_strategy_choose[i][0][-3:-1])-1].append(random_strategy_choose[i][0][2:-4])
        else: #注射時間是個位數
            new_strategy[int(random_strategy_choose[i][0][-2])-1].append(random_strategy_choose[i][0][2:-3])

#     new_strategy = list(map(int, new_strategy)) #str-->int

    temp = "Y"
    for i in range(len(new_strategy)): #如果違反Budget
        if(len(new_strategy[i]) > B):
            temp = "N"
            break
    if(temp=="N"):
        continue
    print("選擇的策略=",new_strategy)
    ########################################################################開始注射策略
    G_temp = copy_graph(G1)
    S = init_S #初始感染點
    temp_vaccinated = []
    temp_infected = [S]
    temp_vulnerable = V.copy()    
    for i in range(len(new_strategy)): #開始依序注射
        if(len(new_strategy[i])==0):#如果這個時間點沒有注射
            a = temp_vulnerable.copy()
            for n in range(len(a)):#病毒開始擴散
                if(G_temp.GetShortPath(S,a[n])==i+1):
                    temp_infected.append(a[n])
                    temp_vulnerable.remove(a[n])
            continue
        TT_saved = []
        ttemp_saved = []
        for j in range(len(new_strategy[i])):
#             temp_saved = []
            b = temp_vulnerable.copy()
            if(int(new_strategy[i][j]) not in temp_vulnerable): #如果要注射的點已經被保護了(從圖中刪除)
#                 for n in range(len(b)):
#                      if(G_temp.GetShortPath(S,b[n])==i+1):#病毒開始擴散
#                         temp_infected.append(b[n])
#                         temp_vulnerable.remove(b[n])
                continue
                

            for k in range(len(temp_vulnerable)):#每一個點計算是否有被注射的點保護

                r  = i+1
                if(r<=(G_temp.GetShortPath(S,temp_vulnerable[k])-G_temp.GetShortPath(int(new_strategy[i][j]),temp_vulnerable[k]))):
                    ttemp_saved.append(temp_vulnerable[k])
        
        temp_saved = list(dict.fromkeys(ttemp_saved))
        for m in range(len(temp_saved)): #調整位置
            temp_vaccinated.append(temp_saved[m])
            temp_vulnerable.remove(temp_saved[m])
            G_temp.DelNode(temp_saved[m])

        a = temp_vulnerable.copy()
        for n in range(len(a)):#病毒開始擴散
            if(G_temp.GetShortPath(S,a[n])==i+1):
                temp_infected.append(a[n])
                temp_vulnerable.remove(a[n])

            
    print("保護個數=",len(temp_vaccinated))
    if(len(temp_vaccinated)>maximum_saved_nodes):
        maximum_saved_nodes = len(temp_vaccinated)
        Final_strategy = new_strategy
    count = count+int(len(temp_vaccinated))
#     print("被保護的點=",temp_vaccinated)
# print("平均被保護個數=",count/timer)
print("Final_strategy = ",Final_strategy)
print("Maximum saved node = ",maximum_saved_nodes)

LP Rounding
round= 0
round= 1
round= 2
round= 3
round= 4
round= 5
round= 6
round= 7
選擇的策略= [['660', '1265'], ['2061'], ['927'], ['1339', '1901'], ['2051'], ['743'], []]
保護個數= 1353
round= 8
選擇的策略= [['912'], ['269'], ['1719'], ['1339', '478'], [], ['743'], []]
保護個數= 936
round= 9
round= 10
round= 11
round= 12
round= 13
round= 14
round= 15
round= 16
round= 17
round= 18
round= 19
round= 20
round= 21
選擇的策略= [['337', '443'], [], ['411'], ['2152', '824'], ['2051', '1132'], ['743'], []]
保護個數= 1334
round= 22
round= 23
round= 24
round= 25
round= 26
round= 27
round= 28
round= 29
round= 30
選擇的策略= [['188'], ['115', '1474'], ['1719'], ['2152'], ['2051', '223'], ['743', '1288'], []]
保護個數= 1070
round= 31
round= 32
round= 33
round= 34
選擇的策略= [['1270', '948'], ['115', '1669'], ['1754', '319'], ['1901'], ['2040', '1132'], ['743', '1288'], []]
保護個數= 1379
round= 35
round= 36
round= 37
round= 38
round= 39
選擇的策略= [[], ['115', '1474'], ['1719'], ['2152', '1896'], ['2051', '2146'], ['743', '1288'], []]
保護個數= 46

In [18]:
end_time = time.time()
print("計算時間=",end_time-start_time,"sec")

計算時間= 54.97227358818054 sec


# Threshold

In [26]:
start_time = time.time()

In [27]:
import random
Threshold = []
temp = 0.01
while(temp < 0.99):
    Threshold.append(temp)
    temp = temp+0.01

Final_threshold = 0
timer = 10
count = 0
maximum_saved_nodes = 0 #計算最多被保護的點
Final_strategy = [] #儲存最終最好的策略
print("Threshold Finding")
for TH in range(len(Threshold)):
    print("Threshold=",Threshold[TH])
    strategy_choose = [] #新策略(格式很亂)
    new_strategy = [[]for i in range(layer)]#新策略(格式清理過)

    for i in range(1,layer+1): #以每一個時間抓出來算機率
        XX = []
        XX_ori = []
        XX_Pro = []

        for x in range(len(THE_X)):
            if THE_X[x][-4]==",":
                if THE_X[x][-3:-1]==str(i):
                    XX.append(THE_X[x])
                    XX_Pro.append(X_Pro[x])
            elif THE_X[x][-2]==str(i):
                XX.append(THE_X[x])
                XX_Pro.append(X_Pro[x])
        if(len(XX)==0): #如果在某時沒有注射
            continue
        XX_ori = XX

        
        for i in range(len(XX_ori)): #判斷threshold #new Threshold
            if(XX_Pro[i]>Threshold[TH]):
                strategy_choose.append([XX[i]])
            
                
#     print("strategy",strategy_choose)#
    #清理格式    
    for i in range(len(strategy_choose)):
        if(strategy_choose[i][0][-4]==","): #注射時間是十位數
            new_strategy[int(strategy_choose[i][0][-3:-1])-1].append(strategy_choose[i][0][2:-4])
        else: #注射時間是個位數
            new_strategy[int(strategy_choose[i][0][-2])-1].append(strategy_choose[i][0][2:-3])

#     new_strategy = list(map(int, new_strategy)) #str-->int

    temp = "Y"
    for i in range(len(new_strategy)): #如果違反Budget
        if(len(new_strategy[i]) > B):
            temp = "N"
            break
    if(temp=="N"):
        continue
    print("選擇的策略=",new_strategy)
    ########################################################################開始注射策略
    G_temp = copy_graph(G1)
    S = init_S #初始感染點
    temp_vaccinated = []
    temp_infected = [S]
    temp_vulnerable = V.copy()    
    for i in range(len(new_strategy)): #開始依序注射
        if(len(new_strategy[i])==0):#如果這個時間點沒有注射
            a = temp_vulnerable.copy()
            for n in range(len(a)):#病毒開始擴散
                if(G_temp.GetShortPath(S,a[n])==i+1):
                    temp_infected.append(a[n])
                    temp_vulnerable.remove(a[n])
            continue
        TT_saved = []
        ttemp_saved = []
        for j in range(len(new_strategy[i])):
#             temp_saved = []
            b = temp_vulnerable.copy()
            if(int(new_strategy[i][j]) not in temp_vulnerable): #如果要注射的點已經被保護了(從圖中刪除)
#                 for n in range(len(b)):
#                      if(G_temp.GetShortPath(S,b[n])==i+1):#病毒開始擴散
#                         temp_infected.append(b[n])
#                         temp_vulnerable.remove(b[n])
                continue
                

            for k in range(len(temp_vulnerable)):#每一個點計算是否有被注射的點保護

                r  = i+1
                if(r<=(G_temp.GetShortPath(S,temp_vulnerable[k])-G_temp.GetShortPath(int(new_strategy[i][j]),temp_vulnerable[k]))):
                    ttemp_saved.append(temp_vulnerable[k])
        
        temp_saved = list(dict.fromkeys(ttemp_saved))
        for m in range(len(temp_saved)): #調整位置
            temp_vaccinated.append(temp_saved[m])
            temp_vulnerable.remove(temp_saved[m])
            G_temp.DelNode(temp_saved[m])

        a = temp_vulnerable.copy()
        for n in range(len(a)):#病毒開始擴散
            if(G_temp.GetShortPath(S,a[n])==i+1):
                temp_infected.append(a[n])
                temp_vulnerable.remove(a[n])

            
    print("保護個數=",len(temp_vaccinated))
    if(len(temp_vaccinated)>maximum_saved_nodes):
        maximum_saved_nodes = len(temp_vaccinated)
        Final_strategy = new_strategy
        Final_threshold = Threshold[TH]
    count = count+int(len(temp_vaccinated))
#     print("被保護的點=",temp_vaccinated)
# print("平均被保護個數=",count/timer)
print("Final_threshold = ",Final_threshold)
print("Final_strategy = ",Final_strategy)
print("Maximum saved node = ",maximum_saved_nodes)

Threshold Finding
Threshold= 0.01
Threshold= 0.02
Threshold= 0.03
Threshold= 0.04
Threshold= 0.05
Threshold= 0.060000000000000005
Threshold= 0.07
Threshold= 0.08
Threshold= 0.09
Threshold= 0.09999999999999999
Threshold= 0.10999999999999999
Threshold= 0.11999999999999998
Threshold= 0.12999999999999998
Threshold= 0.13999999999999999
Threshold= 0.15
Threshold= 0.16
Threshold= 0.17
Threshold= 0.18000000000000002
Threshold= 0.19000000000000003
Threshold= 0.20000000000000004
Threshold= 0.21000000000000005
Threshold= 0.22000000000000006
Threshold= 0.23000000000000007
Threshold= 0.24000000000000007
Threshold= 0.25000000000000006
Threshold= 0.26000000000000006
Threshold= 0.2700000000000001
Threshold= 0.2800000000000001
Threshold= 0.2900000000000001
Threshold= 0.3000000000000001
Threshold= 0.3100000000000001
Threshold= 0.3200000000000001
Threshold= 0.3300000000000001
選擇的策略= [['188', '1270'], ['1474'], ['1719', '927'], ['2152', '478'], ['2051', '2040'], ['743', '1288'], []]
保護個數= 1427
Threshold= 

In [28]:
end_time = time.time()
print("計算時間=",end_time-start_time,"sec")

計算時間= 185.8737506866455 sec
